In [1]:
#general 
import pandas as pd
import numpy as np
import os
from sklearn import __version__ as sklearn_version
from sklearn.model_selection import train_test_split

#warnings 
import warnings 
warnings.filterwarnings("ignore")

#for RNN
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from keras.models import load_model

import pickle

import ast

Create Sentences for testing RNNS

In [2]:
year = "2019"

In [3]:
word_df = pd.read_pickle("pos_tag_word_list")
sent_df = pd.read_csv("sample sent idea.csv")

Develop Benchmarks

In [4]:
def benchmark_string(sents):
    list_of_strings = []

    for word_list in sents:
        list_of_strings.append(' '.join(word_list))
    return list_of_strings

In [5]:
sent_df['list'] = sent_df['list'].apply(ast.literal_eval)
benchmark_sentences = sent_df['list'].tolist()
all_benchmarks = benchmark_string(benchmark_sentences)

In [6]:
loaded_model = tf.keras.models.load_model("C:/Users/danie/Desktop/RNN_"+year)

In [7]:
X_b = all_benchmarks
x_arrayb = np.asarray(X_b)
y_predb = loaded_model.predict(X_b)
Yb = pd.DataFrame(y_predb)
Yb.reset_index(inplace=True, drop=True)
Yb.set_axis(['0 Class Benchamark', '1 Class Benchmark'], axis='columns', inplace=True)
sent_df = pd.concat([sent_df, Yb], axis=1)

5/5 [==============================] - 4s 16ms/step


Test RNN model

#NOUNS

In [8]:
sent_df['replacement string'] = sent_df['replacement string'].apply(ast.literal_eval)


noun_list = word_df[(word_df.pos_tag_short == "N")]["feature"].values.tolist()
verb_list = word_df[(word_df.pos_tag_short == "V")]["feature"].values.tolist()
adverb_list = word_df[(word_df.pos_tag_short == "R")]["feature"].values.tolist()
adjective_list = word_df[(word_df.pos_tag_short == "J")]["feature"].values.tolist()

noun_sent = sent_df[(sent_df['POS tag'] == "Noun")]["replacement string"].values.tolist()
verb_sent = sent_df[(sent_df['POS tag'] == "Verb")]["replacement string"].values.tolist()
adjective_sent = sent_df[(sent_df['POS tag'] == "Adjective")]["replacement string"].values.tolist()
adverb_sent = sent_df[(sent_df['POS tag'] == "Adverb")]["replacement string"].values.tolist()

adverb_len = len(adverb_sent)
adjective_len = len(adjective_sent)
noun_len = len(noun_sent)
verb_len = len(verb_sent)

In [9]:
def word_list(sublist, sentences):
    output=[]
    for sentence in sentences:
        index=sentence.index('UNK')
        sentence_copy = sentence.copy()
        for sub in sublist:
            sentence_copy[index]=sub
            output.append([' '.join(sentence_copy.copy()),sentence])

    return output



sent_n = pd.DataFrame(word_list(noun_list, noun_sent))
sent_j = pd.DataFrame(word_list(adjective_list, adjective_sent))
sent_a = pd.DataFrame(word_list(adverb_list, adverb_sent))
sent_v = pd.DataFrame(word_list(verb_list, verb_sent))

In [10]:
x_array_n = np.asarray(sent_n[0])
x_array_j = np.asarray(sent_j[0])
x_array_v = np.asarray(sent_v[0])
x_array_a = np.asarray(sent_a[0])


y_pred_n = loaded_model.predict(x_array_n)
y_pred_j = loaded_model.predict(x_array_j)
y_pred_v = loaded_model.predict(x_array_v)
y_pred_a = loaded_model.predict(x_array_a)

1/1 [==============================] - 0s 32ms/step


In [11]:
#You need to change the num_repititons to the number of sentences in the function

def labels(word_list, word_len):
    num_repetitions = word_len
    repeated_words = []
    repeated_words = np.tile(word_list, word_len)
    return repeated_words

noun_labels = pd.DataFrame(labels(noun_list, noun_len))
adjective_labels = pd.DataFrame(labels(adjective_list, adjective_len))
verb_labels = pd.DataFrame(labels(verb_list, verb_len))
adverb_labels = pd.DataFrame(labels(adverb_list, adverb_len))



In [12]:
Y_n = pd.DataFrame(y_pred_n)
Y_j = pd.DataFrame(y_pred_j)
Y_v = pd.DataFrame(y_pred_v)
Y_a = pd.DataFrame(y_pred_a)

result_n = pd.concat([noun_labels, Y_n,sent_n], axis=1)
result_j = pd.concat([adjective_labels, Y_j,sent_j], axis=1)
result_v = pd.concat([verb_labels, Y_v,sent_v], axis=1)
result_a = pd.concat([adverb_labels, Y_a, sent_a], axis=1)

result_n.set_axis(['feature', '0 Class', '1 Class', 'RNN Sentences', 'replacement string'], axis='columns', inplace=True)
result_j.set_axis(['feature', '0 Class', '1 Class', 'RNN Sentences', 'replacement string'], axis='columns', inplace=True)
result_v.set_axis(['feature', '0 Class', '1 Class', 'RNN Sentences', 'replacement string'], axis='columns', inplace=True)
result_a.set_axis(['feature', '0 Class', '1 Class', 'RNN Sentences', 'replacement string'], axis='columns', inplace=True)

In [13]:
result = pd.concat([result_a, result_v, result_j, result_n])

In [14]:
result_with_source = pd.merge(result, word_df, on='feature', how='left')
#verb problem

In [15]:
result_with_source = result_with_source.dropna()
result_with_source['replacement string'] = result_with_source['replacement string'].apply(tuple)
sent_df['replacement string'] = sent_df['replacement string'].apply(tuple)

In [16]:
df = pd.merge(result_with_source, sent_df, on='replacement string', how='inner')

In [17]:
df['Class 0 Differential'] = df['0 Class'] - df['0 Class Benchamark']
df['Class 1 Differential'] = df['1 Class'] - df['1 Class Benchmark']


In [18]:
df

,feature,0 Class,1 Class,RNN Sentences,replacement string,source,pos_tag,pos_tag_short,snet id,list,sent,POS tag,word num,identified word,0 Class Benchamark,1 Class Benchmark,Class 0 Differential,Class 1 Differential
0,friendly,0.126203,-0.199339,say guy hear sing think moyles year friendly,"(say, guy, hear, sing, think, moyles, year, UNK)",communality,RB,R,7551,"[say, guy, hear, sing, think, moyles, year, ago]","He said: ""Some guys heard me singing Tom Jones...",Adverb,7,ago,0.138972,-0.247487,-0.012768,0.048148
1,just,0.126780,-0.242009,say guy hear sing think moyles year just,"(say, guy, hear, sing, think, moyles, year, UNK)",communality,RB,R,7551,"[say, guy, hear, sing, think, moyles, year, ago]","He said: ""Some guys heard me singing Tom Jones...",Adverb,7,ago,0.138972,-0.247487,-0.012192,0.005478
2,sincere,0.372808,-0.420976,say guy hear sing think moyles year sincere,"(say, guy, hear, sing, think, moyles, year, UNK)",communality,RB,R,7551,"[say, guy, hear, sing, think, moyles, year, ago]","He said: ""Some guys heard me singing Tom Jones...",Adverb,7,ago,0.138972,-0.247487,0.233837,-0.173489
3,cyberbully,0.144392,-0.281026,say guy hear sing think moyles year cyberbully,"(say, guy, hear, sing, think, moyles, year, UNK)",Violence,RB,R,7551,"[say, guy, hear, sing, think, moyles, year, ago]","He said: ""Some guys heard me singing Tom Jones...",Adverb,7,ago,0.138972,-0.247487,0.005421,-0.033539
4,unruly,-0.370504,0.430185,say guy hear sing think moyles year unruly,"(say, guy, hear, sing, think, moyles, year, UNK)",Violence,RB,R,7551,"[say, guy, hear, sing, think, moyles, year, ago]","He said: ""Some guys heard me singing Tom Jones...",Adverb,7,ago,0.138972,-0.247487,-0.509476,0.677672
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54335,slender,3.167279,-3.804033,terrify slender play well together also score ...,"(terrify, UNK, play, well, together, also, sco...",Apperance,NN,N,9512,"[terrify, pace, play, well, together, also, sc...",Aaron Lennon terrified the Croats with his pac...,Noun,1,pace,3.165017,-3.799248,0.002262,-0.004785
54336,handsome,3.266845,-3.983990,terrify handsome play well together also score...,"(terrify, UNK, play, well, together, also, sco...",Apperance,NN,N,9512,"[terrify, pace, play, well, together, also, sc...",Aaron Lennon terrified the Croats with his pac...,Noun,1,pace,3.165017,-3.799248,0.101828,-0.184742
54337,fat,3.516447,-4.136196,terrify fat play well together also score twic...,"(terrify, UNK, play, well, together, also, sco...",Apperance,NN,N,9512,"[terrify, pace, play, well, together, also, sc...",Aaron Lennon terrified the Croats with his pac...,Noun,1,pace,3.165017,-3.799248,0.351431,-0.336948
54338,thin,2.302521,-2.783316,terrify thin play well together also score twi...,"(terrify, UNK, play, well, together, also, sco...",Apperance,NN,N,9512,"[terrify, pace, play, well, together, also, sc...",Aaron Lennon terrified the Croats with his pac...,Noun,1,pace,3.165017,-3.799248,-0.862496,1.015932


In [19]:
file_name = "RNN_Test_Sentences_"+year

with open(file_name, 'wb') as handle:
    pickle.dump(df, handle, protocol=pickle.HIGHEST_PROTOCOL)